# Whole-Brain Ergebniskarten mit HRF als Modell

### module importieren

In [ ]:
import numpy as np
from nilearn import image, input_data, plotting
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from nistats import hemodynamic_models

sns.set_context('poster')

### Dateiname des funktionellen Bildes

In [ ]:
epi_file = '../data/sub-01_task-imagery_run-02_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

### Extraktion der Daten aus der Region

Whole-Brain Maske definieren

In [ ]:
my_standard = True
my_detrend = True
my_smoo = 8

In [ ]:
brain_masker = input_data.NiftiMasker(mask_img='../data/sub-01_task-imagery_run-02_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz',
                                            standardize=my_standard,
                                            detrend=my_detrend,
                                            smoothing_fwhm=my_smoo).fit()

Daten auslesen

In [ ]:
brain_data = brain_masker.transform(epi_file)

Daten in Tabelle packen

In [ ]:
brain_df = pd.DataFrame(brain_data)

### Studiendesign laden

In [ ]:
design_df = pd.read_csv('../data/sub-01_task-imagery_run-02_events.tsv',sep='\t',index_col=[2,4])
design_df = design_df.sort_index()
design_df

### Modell der hämodynamischen Antwort

In [ ]:
my_tr = 3.0
my_onset = 0.0
my_length = 32.0

In [ ]:
hrf_model = hemodynamic_models.spm_hrf(tr=my_tr,
                                   oversampling=my_tr,
                                   time_length=my_length+my_onset,
                                   onset=my_onset)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(hrf_model,'-o',label='erwartete messbare Antwort')
plt.axvline(my_onset,color='r',label='neuronales Feuern')
plt.axhline(0,color='k')
sns.despine()
plt.legend(loc=(1,0.5))
plt.xlabel('Zeit in Sekunden')
plt.show()

### Design und HRF miteinander verbinden

In [ ]:
number_of_volumes = image.load_img(epi_file).shape[-1]
number_of_volumes

In [ ]:
def make_onsets(df,number_of_volumes=number_of_volumes,my_tr=my_tr):
    design = np.zeros(int(number_of_volumes*my_tr))
    # für jede Zeile in der Tabelle
    for start in range(df.shape[0]):
        # Inhalte der Zeile
        block = df.iloc[start]
        # Onsets und Durations extrahieren
        block_onset = block['onset']
        block_duration = block['duration']
        # Ab Onset so viele 1en einfügen wie der Block lang ist
        for event in range(block_duration):
            design[block_onset+event] = 1.
    return design

In [ ]:
def my_design(my_condition):
    
    beispiel_design = make_onsets(design_df.loc[my_condition,['onset','duration']])
    hrf_design_in_tr = np.convolve(beispiel_design,hrf_model)[::int(my_tr)]
    
    return hrf_design_in_tr

In [ ]:
hrf_design_in_tr = my_design('MOTORIK')

In [ ]:
plt.plot(hrf_design_in_tr);

### Design für alle Bedingungen

In [ ]:
# Leere Tabelle in die wir hineinschreiben
design_matrix_df = pd.DataFrame()
# für jede der 5 Bedingungen aus design_df
for condition in design_df.index.levels[0]:
    # verbinde das design mit der hrf
    hrf_design_in_tr = my_design(condition)
    # mach daraus eine tabelle
    hrf_df = pd.DataFrame(hrf_design_in_tr)
    # der spaltenname der tabelle entspricht dem namen der bedingung
    hrf_df.columns = [condition]
    # die akuelle tabelle wird der großen tabelle angehängt
    design_matrix_df = pd.concat([design_matrix_df,hrf_df],axis=1)

In [ ]:
design_matrix_df.tail()

Visualisierung mit der Zeit auf der x-Achse

In [ ]:
plt.figure(figsize=(16,6))
for condition in design_matrix_df.columns:
    plt.plot(design_matrix_df.loc[:,condition],label=condition)
sns.despine(trim=True)
plt.legend(loc=(0.95,0.3))
plt.xlabel('time in volumes')
plt.ylabel('predicted activity\n(arbitrary units)')
plt.show()

Alternative Ansicht "von oben" als Heatmap (Zeit auf der y-Achse)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(7,9))
sns.heatmap(design_matrix_df,
            cbar_kws={'label': 'predicted activity\n(arbitrary units)'},
            cmap='Greys_r',
            ax=ax)
plt.ylabel('time in volumes')
plt.show()

### Korrelationen rechnen

Bedingung auswählen

In [ ]:
my_onsets = design_matrix_df.loc[:,'MOTORIK']

In [ ]:
my_onsets.plot();

Korrelationen des Signalvarlaufs jedes Voxels mit der Bedingung rechnen

In [ ]:
corr_data = brain_df.corrwith(my_onsets)

Nachdem der Zeitverlauf (203 Volumen) jedes Voxels mit dem HRF-Modell korreliert wurde, gibt es so viele Korrelationen wie es Voxel gibt:

In [ ]:
corr_data.shape

Korrelationen in Hirnbild umwandeln (den Korrelationswert jedes Voxels an die entsprechende Stelle im 3D-Hirnraum packen)

In [ ]:
corr_img = brain_masker.inverse_transform(corr_data)

In [ ]:
type(corr_img)

Hirnbild visualisieren

In [ ]:
plotting.plot_stat_map(corr_img,
                       threshold=0.3,
                       display_mode='z',
                       cut_coords=8,
                       bg_img='../data/sub-01_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz');